# 1. Import Libraries

In [48]:
import splitfolders
import tensorflow as tf
import warnings
warnings.simplefilter(action='ignore', category=Warning)

from sklearn.metrics import classification_report

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Dense, \
GlobalAveragePooling2D, Lambda
from tensorflow.keras.optimizers import RMSprop 
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50  import preprocess_input

# 2. Split Data into Train/Val/Test Folders

In [2]:
splitfolders.ratio(
    "./data_by_class/", 
    output="./data_by_set/", 
    seed=21, 
    ratio=(0.9, 0.05, 0.05), 
    group_prefix=None
)

Copying files: 24955 files [04:27, 93.14 files/s] 


# 3. Prepare Image Augmentation

In [49]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=10,
    brightness_range=[0.80,1.20],
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# 4. Prepare Generator Data Set

In [50]:
train_dataset = train_datagen.flow_from_directory(
    directory='./data_by_set/train',
    target_size=(150, 150), 
    batch_size=64,
    shuffle=False,
    class_mode='binary',
    seed=21
)

valid_dataset = valid_datagen.flow_from_directory(
    directory='./data_by_set/val',
    target_size=(150, 150), 
    batch_size=32,
    shuffle=False,
    class_mode='binary',
    seed=21
)

test_dataset = test_datagen.flow_from_directory(
    directory='./data_by_set/test',
    target_size=(150, 150), 
    batch_size=32,
    shuffle=False,
    class_mode='binary',
    seed=21
)

Found 22457 images belonging to 2 classes.
Found 1247 images belonging to 2 classes.
Found 1249 images belonging to 2 classes.


# 5. Initialize and Train Model (Some Weights Unfreezed)

In [51]:
pre_trained_model = ResNet50(input_shape=(150,150,3),
                                include_top=False,
                                weights="imagenet")

# Some weights in later layers are unfreezed
for layer in pre_trained_model.layers[:-5]:
    layer.trainable=False

In [52]:
tf.random.set_seed(21)

model = tf.keras.models.Sequential([
    pre_trained_model,
    GlobalAveragePooling2D(),    
    Dense(512,activation="swish"),
    Dropout(0.7),
    Dense(256,activation="swish"),
    Dropout(0.5),
    Dense(128,activation="swish"),
    Dropout(0.3), 
    Dense(32,activation="tanh"),
    Dropout(0.2), 
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [53]:
%%time

history = model.fit(
    train_dataset,
    steps_per_epoch=200,
    epochs=2,
    validation_data=valid_dataset,  
)

Epoch 1/2
200/200 [==============================] - 755s 4s/step - loss: 0.6432 - accuracy: 0.6446 - val_loss: 0.1101 - val_accuracy: 0.9639
Epoch 2/2
200/200 [==============================] - 833s 4s/step - loss: 0.2349 - accuracy: 0.9172 - val_loss: 0.2558 - val_accuracy: 0.9038
Wall time: 26min 28s


# 6. Evaluate on Test Set

In [54]:
test_labels = test_dataset.classes
predictions=model.predict_generator(test_dataset)
predictions = predictions.reshape(1,-1)[0]>0.5

print(classification_report(test_labels, predictions,target_names=['Cat','Dog']))

              precision    recall  f1-score   support

         Cat       0.92      0.92      0.92       625
         Dog       0.92      0.92      0.92       624

    accuracy                           0.92      1249
   macro avg       0.92      0.92      0.92      1249
weighted avg       0.92      0.92      0.92      1249

